# Training

In [ ]:
import torch.nn as nn
from model import Resnet50FPN, CountRegressor, weights_normal_init, weights_xavier_init
from utils_ltce import MAPS, Scales, Transform,TransformTrain,extract_features, visualize_output_and_save
from PIL import Image
import os
import torch
import argparse
import json
import numpy as np
from tqdm import tqdm
from os.path import exists,join
import random
import torch.optim as optim
import torch.nn.functional as F
import cv2
import matplotlib.pyplot as plt
from collections import defaultdict
import torchvision.ops.boxes as bops
from funcs import *

In [ ]:
data_path = '/content/drive/MyDrive/LearningToCountEverything_new/data/'
output_dir = "./logsSave"
test_split = "train" #choices=["train", "test", "val"]
gpu = 0 
learning_rate = 1e-5
anno_file = data_path + 'annotation_FSC147_384.json'
data_split_file = data_path + 'Train_Test_Val_FSC_147.json'
im_dir = data_path + 'images_384_VarV2'
gt_dir = data_path + 'gt_density_map_adaptive_384_VarV2'
pre_trained_backbone = 'resnet' #choices=[resnet,wide_resnet,vgg16]
regressor = CountRegressor(6, pool='mean')
optimizer = optim.Adam(regressor.parameters(), lr = 1e-7)
criterion = nn.MSELoss().cuda()
backbone_model = Resnet50FPN()

In [ ]:
if not exists(output_dir):
    os.mkdir(output_dir)

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu)

#criterion = nn.MSELoss().cuda()
criterion = nn.MSELoss()

#backbone.cuda()
#backbone.eval()

weights_normal_init(regressor, dev=0.001)
regressor.train()
#regressor.cuda()
optimizer = optim.Adam(regressor.parameters(), lr = learning_rate)

with open(anno_file) as f:
    annotations = json.load(f)

with open(data_split_file) as f:
    data_split = json.load(f)

In [ ]:
model_yolo = torch.hub.load('ultralytics/yolov5', 'yolov5l6', pretrained=True)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
%%time
%matplotlib inline
run_train_phase(epochs=3, backbone_model=backbone_model, regressor=regressor, yolo_model=model_yolo,
                optimizer=optimizer, criterion=criterion, annotations=annotations, 
                data_train=data_split['train'], shuffle=False, data_val=data_split['val'], 
                num_img_train=5, num_img_val=5, yolo_flag=True, yolo_threshold=3, plot_flag=True, 
                save=False, im_dir=data_path+'images_384_VarV2', gt_dir=gt_dir)

# Testing

import torch.nn as nn
from model import Resnet50FPN, CountRegressor, weights_normal_init, weights_xavier_init
from utils_ltce import MAPS, Scales, Transform,TransformTrain,extract_features, visualize_output_and_save
from PIL import Image
import os
import torch
import argparse
import json
import numpy as np
from tqdm import tqdm
from os.path import exists,join
import random
import torch.optim as optim
import torch.nn.functional as F
import cv2
import matplotlib.pyplot as plt
from collections import defaultdict
import torchvision.ops.boxes as bops
from funcs import *

In [ ]:
data_path = '/content/drive/MyDrive/LearningToCountEverything_new/data/'
output_dir = "./logsSave"
test_split = "train" #choices=["train", "test", "val"]
gpu = 0 
learning_rate = 1e-5
anno_file = data_path + 'annotation_FSC147_384.json'
data_split_file = data_path + 'Train_Test_Val_FSC_147.json'
im_dir = data_path + 'images_384_VarV2'
gt_dir = data_path + 'gt_density_map_adaptive_384_VarV2'
pre_trained_backbone = 'resnet' #choices=[resnet,wide_resnet,vgg16]
regressor = CountRegressor(6, pool='mean')
optimizer = optim.Adam(regressor.parameters(), lr = 1e-7)
criterion = nn.MSELoss().cuda()
backbone_model = Resnet50FPN()

In [ ]:
if not exists(output_dir):
    os.mkdir(output_dir)

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu)

#criterion = nn.MSELoss().cuda()
criterion = nn.MSELoss()

#backbone.cuda()
#backbone.eval()

weights_normal_init(regressor, dev=0.001)
regressor.train()
#regressor.cuda()
optimizer = optim.Adam(regressor.parameters(), lr = learning_rate)

with open(anno_file) as f:
    annotations = json.load(f)

with open(data_split_file) as f:
    data_split = json.load(f)

In [ ]:
model_yolo = torch.hub.load('ultralytics/yolov5', 'yolov5l6', pretrained=True)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
%%time
%matplotlib inline
test(data_split['test'][100:], num_img=50, backbone_model=backbone_model, regressor=regressor, 
     yolo_model=model_yolo, yolo_flag=False, yolo_threshold=3, annotations=annotations, 
     plot_flag=True, im_dir=data_path+'images_384_VarV2', use_gpu=False, 
     model_path='/content/drive/MyDrive/LearningToCountEverything_new/data/pretrainedModels/FamNet_Save1.pth',
     adapt=False, gradient_steps=100, learning_rate=1e-7)